### DA & FMIRL

- use same class as pair to avoid extra distance computation
- potential in $Q$ space: $U_{ab}(r)=\sum_Q\frac{f_a(Q)f_b(Q)}{{\overline {f(Q)}}^2}U_Qj_0(Qr)$
    - DA: $U_Q\leftarrow W_Q [S_C(Q)-S_E(Q)]$
    - FMIRL: $U_Q \leftarrow (1-\eta /W_Q)U_Q+\eta [S_C(Q)-S_E(Q)]$
    - L2 is $1/W_Q$

- let $V_Q=U_Q/W_Q$, then $U_{ab}(r)=\sum_Q\frac{f_a(Q)f_b(Q)}{{\overline {f(Q)}}^2}W_QV_Qj_0(Qr)$
    - DA: $V_Q\leftarrow [S_C(Q)-S_E(Q)]$
    - FMIRL:  $V_Q \leftarrow (1-\eta /W_Q)V_Q+\eta/W_Q [S_C(Q)-S_E(Q)]$
        - we further let $\eta=\gamma W_Q$, which will not change the fitting
        - $V_Q \leftarrow (1-\gamma)V_Q+\gamma [S_C(Q)-S_E(Q)]=V_Q+\gamma [S_C(Q)-S_E(Q)-V_Q]$

### Neural network potential

- energy $E=\sum_i E_i$ with $$E_i=A\left(\xi,\sum_j g(\theta_{ij}^k,r_{ij})\right)$$
- force $$\mathbf F_{i}=-\sum_{j\ne i}\left(\frac {\partial E}{\partial g_j^k}+\frac {\partial E}{\partial g_i^k}\right)\frac{\partial g_{ij}^k}{\partial r_{ij}} \hat {\mathbf r} _{ij}
=-\sum_{j}\frac {\partial E}{\partial g_j^k}\left((1-\delta_{ij})\frac{\partial g_{ij}^k}{\partial r_{ij}} \hat {\mathbf r} _{ij} + \delta_{ij}\sum_{l\ne i} \frac{\partial g_{il}^k}{\partial r_{il}} \hat {\mathbf r} _{il}\right)$$
    - instantanious pair force
- implementation:
    - round 1: compute $\sum_j\mathbf g(\theta,\mathbf r_i;\mathbf r_j)$, store result in GLOBAL memory
    - round 2: call GPU to compute $\partial_{\mathbf g}A$ (and $A$)
    - round 3: compute force

### Derivitive of parameters

- loss $$L=\sum_i(\mathbf F_i-\mathbf F_{i,0})^2 + \alpha (E-E_0)^2$$

- force field potential $u(\theta,g(\mathbf r_1,\mathbf r_2,\cdots))$:
    - energy $$\partial_\theta E=\sum\partial_\theta u(\theta,g(\mathbf r_1,\mathbf r_2,\cdots))$$
    - force $$\partial_\theta \mathbf F_i=-\sum\partial_\theta\partial_g u(\theta,g) \partial_{\mathbf r_i }g$$

- NNP potential:
    - define $$D_{ij}^k=(1-\delta_{ij})\frac{\partial g_{ij}^k}{\partial r_{ij}} \hat {\mathbf r} _{ij} + \delta_{ij}\sum_{l\ne i} \frac{\partial g_{il}^k}{\partial r_{il}} \hat {\mathbf r} _{il}$$
    - then everything can be implemented in TF (otherwise, a Jacobian must be returned by TF)
    - Force (though this might not be efficient because $D$ is sparse) $$\mathbf F_{i}=-\sum_{jk}\frac {\partial E}{\partial g_j^k} D_{ij}^k$$
    - losses can be computed from force
    - gradient (not used explicitly because included in automatic diffrenciation) $$\partial_\xi L_F = -2\sum_i\sum_{j\ne i}(\mathbf F_i-\mathbf F_{i,0}-(\mathbf F_j-\mathbf F_{j,0}))\cdot\hat {\mathbf r} _{ij}\frac{\partial g_{ij}^k}{\partial r_{ij}}\left(\frac {\partial^2 E}{\partial \xi\partial g_i^k}\right)$$

- gradient of $\theta$ is computed on cpu $$\partial_\theta L_E = 2(E-E_0)\sum_i\frac{\partial E}{\partial g_i^k}\frac{\partial g_i^k}{\partial \theta^k}$$
$$\partial_\theta L_F = -2\sum_i\sum_{j\ne i}(\mathbf F_i-\mathbf F_{i,0})\cdot\hat {\mathbf r} _{ij}\left(\frac {\partial E}{\partial g_j^k}+\frac {\partial E}{\partial g_i^k}\right)\frac{\partial^2 g_{ij}^k}{\partial \theta ^k\partial r_{ij}}$$
$$\partial_\theta L = 2\sum_i \frac{\partial E}{\partial g_i^k} \left\{(E-E_0)\sum_{j\ne i}\frac{\partial g_{ij}^k}{\partial \theta^k}+\sum_{j\ne i}(\mathbf F_{i,0}-\mathbf F_i-(\mathbf F_{j,0}-\mathbf F_j))\cdot\hat {\mathbf r} _{ij}\frac{\partial^2 g_{ij}^k}{\partial \theta^k\partial r_{ij}}\right\}$$